In [1]:
import sys
import os
import pandas as pd

# Caminho até a raiz do projeto (a pasta que contém "titulospub/")
caminho_raiz = os.path.abspath("Z:\\Chila\\projetos\\calculadora_titulos_publicos")
if caminho_raiz not in sys.path:
    sys.path.append(caminho_raiz)


from titulospub import *

In [2]:
vm = VariaveisMercado()
vm.atualizar_tudo()

🔄 Atualizando variáveis de mercado...
📡 Buscando feriados via scraping...
📡 Calculando IPCA dict...
📡 Buscando CDI...
📡 Realizando scraping ANBIMA...
♻️ Cache salvo para todos os títulos ANBIMA.
📡 Realizando scraping BMF...
♻️ Cache salvo para todos os contrados de DI e DAP.
📡 Realizando scraping VNA_LFT...
♻️ Cache salvo para VNA_LFT.
✅ Atualização concluída.


In [3]:
vm._vna_lft

17353.385318

In [25]:
di_df = vm._bmf["DI"]
di_df.head(3)

,DATA,DATA_VENCIMENTO,DI,ADJ
0,2025-09-08,2025-10-01,DI1V25,14.904
1,2025-09-08,2025-11-03,DI1X25,14.906
2,2025-09-08,2025-12-01,DI1Z25,14.903


In [6]:
vm._anbimas["NTN-B"]

,TITULO,DATA,VENCIMENTO,ANBIMA,PU
0,NTN-B,2025-09-11,2026-08-15,9.5223,4422.787870
1,NTN-B,2025-09-11,2027-05-15,8.4846,4461.452219
2,NTN-B,2025-09-11,2028-08-15,7.9385,4342.515000
3,NTN-B,2025-09-11,2029-05-15,7.7696,4388.433168
4,NTN-B,2025-09-11,2030-08-15,7.7488,4256.594223
5,NTN-B,2025-09-11,2032-08-15,7.7170,4166.363798
6,NTN-B,2025-09-11,2033-05-15,7.6697,4213.788075
7,NTN-B,2025-09-11,2035-05-15,7.5674,4169.330651
8,NTN-B,2025-09-11,2040-08-15,7.3156,4048.551624
9,NTN-B,2025-09-11,2045-05-15,7.2846,4052.112755


In [26]:
di_df["DIAS_VENCIMENTO"] = di_df.apply(lambda row: dias_trabalho_total(
                                                                     pd.Timestamp.today().normalize(),
                                                                     row["DATA_VENCIMENTO"]
                                                                      ),axis=1)

In [44]:
from scipy.interpolate import CubicSpline
import pandas as pd
from titulospub.utils.datas import dias_trabalho_total

def spline_cubica(dias: any, curva_di_df: pd.DataFrame = None, di_lim: str = None):
    """
    Função para interpolação cúbica da curva DI
    
    Args:
        dias: Lista de dias para interpolação ou valor único
        curva_di_df: DataFrame com a curva DI (opcional)
        di_lim: Limite da curva DI (opcional)
    
    Returns:
        Lista com as taxas interpoladas
    """
    
    if curva_di_df is None:
        from titulospub import VariaveisMercado
        curva_di_df = VariaveisMercado().get_bmf()["DI"]
    
    if di_lim is None:
        # Corrigido: pega o valor único da Series
        di_lim = curva_di_df.tail(1)["DI"].iloc[0]
        
    di_df = curva_di_df.copy()
    di_df["DIAS_VENCIMENTO"] = di_df.apply(lambda row: dias_trabalho_total(
                                                                     pd.Timestamp.today().normalize(),
                                                                     row["DATA_VENCIMENTO"]
                                                                      ), axis=1)

    # Corrigido: verifica se di_lim existe na coluna DI
    if di_lim not in di_df["DI"].values:
        raise ValueError(f"DI limit '{di_lim}' não encontrado na curva")
    
    di_df_corte = di_df[di_df.index <= di_df[di_df["DI"] == di_lim].index[0]]

    x = di_df_corte["DIAS_VENCIMENTO"]
    y = di_df_corte["ADJ"]

    interpolacao = CubicSpline(x=x, y=y, bc_type='natural')

    taxas_resultado = []

    # Pega o último valor da curva (maior dia de vencimento)
    max_dias_curva = di_df_corte["DIAS_VENCIMENTO"].max()
    taxa_ultima_curva = di_df_corte.loc[di_df_corte["DIAS_VENCIMENTO"] == max_dias_curva, "ADJ"].iloc[0]

    # Converte para lista se for um único valor
    if isinstance(dias, (float, int)):
        dias = [dias]

    for dia in dias:
        if dia >= max_dias_curva:
            # Se o dia for maior que o máximo da curva, usa o último valor
            taxas_resultado.append(taxa_ultima_curva)
        else:
            # Se estiver dentro da curva, usa a interpolação
            taxa_interpolada = interpolacao(dia)
            taxas_resultado.append(taxa_interpolada)

    return taxas_resultado


In [45]:
# Teste da função corrigida
try:
    resultado = spline_cubica(dias=[1793, 2408, 3317])
    print("✅ Função executada com sucesso!")
    print(f"Resultado: {resultado}")
except Exception as e:
    print(f"❌ Erro: {e}")
    import traceback
    traceback.print_exc()

✅ Usando cache existente de BMF completo.
✅ Função executada com sucesso!
Resultado: [array(13.65381258), array(13.69919554), array(13.55954347)]


In [46]:
# Teste adicional com parâmetros específicos
try:
    # Teste com di_lim específico
    resultado_lim = spline_cubica(dias=[1070, 2005, 3150], di_lim="DI1F35")
    print("✅ Teste com di_lim executado com sucesso!")
    print(f"Resultado com di_lim: {resultado_lim}")
    
    # Teste com valor único
    resultado_unico = spline_cubica(dias=2000)
    print("✅ Teste com valor único executado com sucesso!")
    print(f"Resultado único: {resultado_unico}")
    
except Exception as e:
    print(f"❌ Erro no teste adicional: {e}")
    import traceback
    traceback.print_exc()


✅ Usando cache existente de BMF completo.
✅ Teste com di_lim executado com sucesso!
Resultado com di_lim: [array(13.3288576), array(13.67652277), np.float64(13.703)]
✅ Usando cache existente de BMF completo.
✅ Teste com valor único executado com sucesso!
Resultado único: [array(13.67507473)]


In [28]:
di_lim = "DI1F35"
di_df_corte = di_df[di_df.index <= di_df[di_df["DI"] == di_lim].index[0]]


In [29]:
di_df[di_df.index <= di_df[di_df["DI"] == di_lim].index[0]]



,DATA,DATA_VENCIMENTO,DI,ADJ,DIAS_VENCIMENTO
0,2025-09-08,2025-10-01,DI1V25,14.904,16
1,2025-09-08,2025-11-03,DI1X25,14.906,39
2,2025-09-08,2025-12-01,DI1Z25,14.903,58
3,2025-09-08,2026-01-02,DI1F26,14.893,80
4,2025-09-08,2026-02-02,DI1G26,14.880,101
5,2025-09-08,2026-03-02,DI1H26,14.835,119
6,2025-09-08,2026-04-01,DI1J26,14.780,141
7,2025-09-08,2026-05-04,DI1K26,14.725,161
8,2025-09-08,2026-06-01,DI1M26,14.629,181
9,2025-09-08,2026-07-01,DI1N26,14.530,202


In [35]:
x = di_df_corte["DIAS_VENCIMENTO"]
y = di_df_corte["ADJ"]

interpolacao = CubicSpline(x=x, y=y, bc_type='natural')


In [69]:
dias = [1070, 2005, 3150]
taxas_resultado = []

# Pega o último valor da curva (maior dia de vencimento)
max_dias_curva = di_df_corte["DIAS_VENCIMENTO"].max()
taxa_ultima_curva = di_df_corte.loc[di_df_corte["DIAS_VENCIMENTO"] == max_dias_curva, "ADJ"].iloc[0]

# Converte para lista se for um único valor
if isinstance(dias, (float, int)):
    dias = [dias]

for dia in dias:
    if dia >= max_dias_curva:
        # Se o dia for maior que o máximo da curva, usa o último valor
        taxas_resultado.append(taxa_ultima_curva)
    else:
        # Se estiver dentro da curva, usa a interpolação
        taxa_interpolada = interpolacao(dia)  # Passa o dia individual, não a lista
        taxas_resultado.append(taxa_interpolada)

taxas_resultado

[array(13.35373635), array(13.72073748), np.float64(13.751)]

In [ ]:
class CURVA_DI:
    def __init__(self, data_vencimento_titulo: str, 
                       data_base: str=None, 
                       dias_liquidacao: int=1,
                       taxa: float=None,
                       quantidade=10000, 
                       cdi: float=None,  
                       feriados: list=None,
                       variaveis_mercado: VariaveisMercado | None = None):